In [11]:
import copy
import cv2
import matplotlib.pyplot as plt
import numpy as np

In [12]:
REGION_NAMES = ['GRASS', 'SIDEWALK', 'BUILDING', 'GRAVEL']

REFERENCE_MISSION = 2

In [13]:
def plot_images(image_list, title_list=[], grid='off'):
    fig, axes = plt.subplots(nrows=1, ncols=len(image_list), figsize=(10*len(image_list), 10*1))
    if len(image_list) == 1:
        axes = [axes]

    for i, ax in enumerate(axes):
        ax.imshow(cv2.cvtColor(image_list[i], cv2.COLOR_BGR2RGB))
        if len(title_list) > 0:
            ax.set_title(title_list[i])
        ax.axis(grid)

In [14]:
def plot_mission_region_sample_images(mission_numbers, mission_images=[], region_images=[], sample_images=[], display_missions=True, display_regions=True, display_samples=True):
    if display_missions:
        if len(mission_images) > 0:
            print('Displaying missions')
            plot_images(mission_images, ['Mission ' + str(mission_number) for mission_number in mission_numbers])
        else:
            print('[Error] No mission images available to display')
    
    if display_regions:
        if len(region_images) > 0:
            print('Displaying regions')
            plot_images(region_images, ['Region ' + str(mission_number) for mission_number in mission_numbers])
        else:
            print('[Error] No region images available to display')
    
    if display_samples:
        if len(sample_images) > 0:
            print('Displaying samples')
            plot_images(sample_images, ['Sample ' + str(mission_number) for mission_number in mission_numbers])
        else:
            print('[Error] No sample images available to display')

In [5]:
def get_mission_file_path(mission_number):
    return '..\\missions\\mission_' + str(mission_number) + '\\mission_' + str(mission_number) + '_'

In [6]:
def get_aligned_global_image(mission_number):
    mission = np.load(get_mission_file_path(mission_number) + 'aligned_image.npy')
    return mission

In [7]:
def get_aligned_region_and_sample_image(mission_number, region_name):
    region = np.load(get_mission_file_path(mission_number) + 'aligned_region_image_' + region_name + '.npy')
    sample = np.load(get_mission_file_path(mission_number) + 'aligned_sample_image_' + region_name + '.npy')
    return region, sample

In [18]:
# load mission images, region images, sample images for given missions and a region
def setup_regions_and_images():
    # select mission numbers and region type
    mission_numbers = [2, 3, 4, 5, 6, 8,9,10,11]
    region_type = 'GRASS'
    mission_images = []
    region_images = []
    sample_images = []

    for mission_number in mission_numbers:
        # load mission images
        mission_image = get_aligned_global_image(mission_number)
        mission_images.append(mission_image)
    
        # load region images and sample images
        region_image, sample_image = get_aligned_region_and_sample_image(mission_number, region_type)
        region_images.append(region_image)
        sample_images.append(sample_image)

    # all lengths and dimensions should match
    #print(len(mission_images))
    #print(mission_images[0].shape)
    #print(len(region_images))
    #print(region_images[0].shape)
    #print(len(sample_images))
    #print(sample_images[0].shape)
    #plot_mission_region_sample_images(mission_numbers, mission_images=mission_images, region_images=region_images, sample_images=sample_images, display_missions=True, display_regions=True, display_samples=True)
    return region_images